In [ ]:
import os
from pathlib import Path

import syft_rds as sy
from syft_core import Client
from syft_rds.server.app import create_app

import syft_flwr

## Set up

In [ ]:
!rm -rf .server .ipynb_checkpoints/

In [ ]:
# reset the simulation folder
!just reset-sim

In [ ]:
EXAMPLE = "quickstart-pytorch"
# EXAMPLE = "embedded-devices"

PROJECT_DIR = Path(os.getcwd()).parent.parent.absolute()
DATASET_DIR = Path("./datasets/").expanduser().absolute()
FLWR_PROJECT_PATH = PROJECT_DIR / "examples" / EXAMPLE
SIM_DIR = PROJECT_DIR / ".sim"
CONFIG_DIR = SIM_DIR / ".config"

if EXAMPLE == "embedded-devices":
    SYFTBOX_DATASET_NAME = "fashionmnist"

elif EXAMPLE == "quickstart-pytorch":
    SYFTBOX_DATASET_NAME = "cifar10"

DO = "yash@openmined.org"
DATASET_NAME = f"{SYFTBOX_DATASET_NAME}_part_1"

print(f"{PROJECT_DIR = }")
print(f"{DATASET_DIR = }")
print(f"{FLWR_PROJECT_PATH = }")
print(f"{SIM_DIR = }")
print(f"{CONFIG_DIR = }")

!just dump-sim-config {DO}

DO_CONFIG = CONFIG_DIR / f"{DO}.json"
!cat $DO_CONFIG

In [ ]:
do_syftbox_client = Client.load(DO_CONFIG)
event_server = create_app(client=do_syftbox_client)
event_server.start()

## DO creates a local dataset

In [ ]:
syft_flwr.utils.save_dataset_to_disk(
    dataset_name=SYFTBOX_DATASET_NAME, dataset_dir=DATASET_DIR, num_partitions=2
)

In [ ]:
do = sy.init_session(host=DO, syftbox_client_config_path=DO_CONFIG)
assert do.host == DO
assert do.email == DO
assert do.is_admin

In [ ]:
DATASET_PRIVATE_PATH = DATASET_DIR / SYFTBOX_DATASET_NAME / DATASET_NAME
DATASET_MOCK_PATH = DATASET_DIR / SYFTBOX_DATASET_NAME / DATASET_NAME
README_PATH = FLWR_PROJECT_PATH / "README.md"

try:
    dataset = do.dataset.create(
        name=SYFTBOX_DATASET_NAME,
        summary=f"Partitioned {SYFTBOX_DATASET_NAME} dataset.",
        description_path=README_PATH,
        path=DATASET_PRIVATE_PATH,
        mock_path=DATASET_MOCK_PATH,
    )
    dataset.describe()
except Exception as e:
    print(f"Error: {e}")

In [ ]:
print(do.datasets)
assert len(do.datasets) == 1

Now DO waits for DS to submit job

## DO Review Jobs

After the DS submits jobs

In [ ]:
jobs = do.jobs.get_all(status="pending_code_review")
assert len(jobs) == 1

job = jobs[-1]

# same as job.code.describe()
job.show_user_code()

os.environ["SYFTBOX_CLIENT_CONFIG_PATH"] = str(DO_CONFIG)
res_job = do.run_private(job)

In [ ]:
do.jobs.share_results(res_job)

## Done

In [ ]:
event_server.stop()

## Run syft_flwr simulation
This can be run after the DS bootstrap the `flwr` project in the `ds.ipynb` notebook

In [ ]:
# flwr_project_path = str(FLWR_PROJECT_PATH)
# mock_path = str(do.dataset.get(name=SYFTBOX_DATASET_NAME).get_mock_path())

# if (FLWR_PROJECT_PATH / "main.py").exists():
#     !just run-sim-client {flwr_project_path} {DO} {mock_path}
# else:
#     print("main.py does not exist in the project")
#     print(
#         "Please wait for the DS to bootstrap the project and then come back to run this cell"
#     )